In [1]:
import torch
from transformers import AutoTokenizer, AutoModel

MODEL_NAME = "Qwen/Qwen3-Embedding-0.6B"

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True
)

model = AutoModel.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    dtype=torch.float16,  # 🔑 FP16
    device_map="auto"           # 🔑 moves model to GPU
)

model.eval()

/Users/viriyadhika/Documents/5.UofT/Projects/SmallAgent/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Qwen3Model(
  (embed_tokens): Embedding(151669, 1024)
  (layers): ModuleList(
    (0-27): 28 x Qwen3DecoderLayer(
      (self_attn): Qwen3Attention(
        (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
        (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
        (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
        (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
        (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
      )
      (mlp): Qwen3MLP(
        (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
        (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
        (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
        (act_fn): SiLUActivation()
      )
      (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
      (post_attention_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
    )
  )
  (norm): Qwen3RM

In [ ]:
from app.chat import MistralGenerator, DeepSeekGenerator, QwenGenerator
from app.common_agent import Router, Agent
from app.coding_agent import DataReaderTool, CodeWriter
from app.pdf_agent import PDFSplitter, PDFPartitioner, Embedder, ChromaStore, PDFIngestor, PDFIngestorTool, QueryAnswererTool, CrossEncoderReranker, RAGRetriever, PDFReaderTool, PDFSplitterTool

def remove_collection():
    temp_store = ChromaStore(path="./chroma_db", collection_name="rag")
    temp_store.client.delete_collection("rag")

def eval(question: str, max_steps: int):
    api_key = ""

    # Create your LLM wrapper
    generator = MistralGenerator(api_key=api_key, temperature=0.1)
    
    generator_dev = MistralGenerator(api_key=api_key, temperature=0.1, model="mistralai/devstral-2512:free")

    # generator = QwenGenerator(api_key=api_key)
    
    splitter = PDFSplitter(pages_per_part=20)
    partitioner = PDFPartitioner()
    embedder = Embedder(tokenizer, model, batch_size=8)

    store = ChromaStore(path="./chroma_db", collection_name="rag")

    reranker = CrossEncoderReranker(
        model_name="cross-encoder/ms-marco-MiniLM-L-6-v2",
        # model_name="cross-encoder/nli-deberta-v3-base",
        top_k=8
    )

    rag = RAGRetriever(
        embedder=embedder,
        vector_store=store,
        reranker=reranker,
        recall_k=20,
    )

    ingestor = PDFIngestor(
        splitter=splitter,
        partitioner=partitioner,
        embedder=embedder,
        store=store
    )

    # Register tools
    tools = [
        CodeWriter(generator_dev),
        DataReaderTool(),
        PDFIngestorTool(ingestor),
        QueryAnswererTool(rag, generator),
        PDFReaderTool(),
        PDFSplitterTool()
    ]

    # Create router / planner
    router = Router(generator, tools)


    # Create agent
    agent = Agent(
        router=router,
        tools=tools,
        max_steps=max_steps
    )

    return agent.run(question)


# 1. Simple Linear regression on data

In [3]:
eval(
    "Do a linear regression from data/data.csv and plot the curve fit, including residual plot and R^2 value",
    12
)

Executing Tool: data_reader with Args {'path': 'data/data.csv'}
Tool Result: ToolResult(artifact_path='data/data_reader1.pkl', result={'info': "<class 'pandas.core.frame.DataFrame'>\nRangeIndex: 7 entries, 0 to 6\nData columns (total 3 columns):\n #   Column                 Non-Null Count  Dtype  \n---  ------                 --------------  -----  \n 0   Zinc grade             7 non-null      float64\n 1   Silver grade           7 non-null      float64\n 2   Error in silver grade  7 non-null      float64\ndtypes: float64(3)\nmemory usage: 300.0 bytes\n", 'columns': {'Zinc grade': 'float64', 'Silver grade': 'float64', 'Error in silver grade': 'float64'}, 'rows': 7})
Executing Tool: code_writer with Args {'task': 'Perform a linear regression on the data, plot the curve fit, including residual plot and R^2 value.', 'context': 'The data is in a pickle file located at data/data_reader1.pkl. The data has the following columns: Zinc grade, Silver grade, Error in silver grade. The data is in 

'The linear regression model has been successfully trained. The R^2 value is 0.9906. The residuals and predictions are saved in the artifact at data/code_writer2.pkl. The artifact contains the trained model, R^2 score, residuals, and predicted values.'

# 2. Linear regression with dirty (with NaN) data

In [4]:
eval(
    "Do a linear regression from data/data_d.csv and plot the curve fit, including residual plot and R^2 value",
    12
)

Executing Tool: data_reader with Args {'path': 'data/data_d.csv'}
Tool Result: ToolResult(artifact_path='data/data_reader1.pkl', result={'info': "<class 'pandas.core.frame.DataFrame'>\nRangeIndex: 7 entries, 0 to 6\nData columns (total 3 columns):\n #   Column                 Non-Null Count  Dtype  \n---  ------                 --------------  -----  \n 0   Zinc grade             7 non-null      float64\n 1   Silver grade           6 non-null      float64\n 2   Error in silver grade  7 non-null      object \ndtypes: float64(2), object(1)\nmemory usage: 300.0+ bytes\n", 'columns': {'Zinc grade': 'float64', 'Silver grade': 'float64', 'Error in silver grade': 'object'}, 'rows': 7})
Executing Tool: code_writer with Args {'task': 'Perform a linear regression on the data in data/data_reader1.pkl, plot the curve fit, including residual plot and R^2 value.', 'context': "The data is in a pandas DataFrame with columns 'Zinc grade', 'Silver grade', and 'Error in silver grade'. The 'Error in silve

"The linear regression analysis has been completed successfully. The R^2 value is 0.9889. The plot of the curve fit, including the residual plot, has been saved as 'linear_regression_plot.png'."

# 3. Read an assignment in a PDF and CSV file and do it

In [5]:
eval(
    "Do the assignment from data/Assignment.pdf the data is available in data/data.csv",
    18
)

Executing Tool: pdf_read with Args {'pdf_path': 'data/Assignment.pdf'}
Tool Result: ToolResult(artifact_path=None, result={'status': 'success', 'content': '[Page 1]\nAssignment: Weighted Least Squares Fitting of Silver Grade vs.\nZinc Grade\nIntroduction\nIn this assignment you will fit a dataset ofsilver grade vs. zinc gradeusing astraight line,\nasecond-order polynomial, and anexponential function. The zinc grade is treated as the\nindependent variable and the silver grade as the dependent variable.\nData\nData is provided in the CSV file\nPart A — Normal Equations Approach for Weighted Least Squares\n(L2)\nLet the weights be defined as\nwi=1\nσ2\ni,\nwhereσ iis the error in the silver grade measurement.\nA1) Weighted Straight Line Fit\nFit the straight line model\ny=a+bx\nusing the weighted normal equations.\n•Report the fitted parametersaandb.\n•Compute and report the RMS error of the residuals.\n•Compute and report the coefficient of determinationR2.\nA2) Weighted Quadratic Fit\nF

"The quadratic model provides a better fit based on both RMS error and R2 score. The quadratic model has a lower RMS error of 0.1035 compared to the linear model's 0.5957, and a higher R2 score of 0.9996 compared to the linear model's 0.9860. The residuals plot is saved to 'data/residuals_plot.png'."

# 4. Read the whole book and answer a question

In [6]:
eval(
    "Read the book in data/some_book.pdf and tell me who try to steal the stone?",
    12
)

{'id': 'gen-1766823699-8JGQPScwCOHla1JMrGLe', 'provider': 'Google', 'model': 'qwen/qwen3-235b-a22b-2507', 'object': 'chat.completion', 'created': 1766823699, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'STOP', 'index': 0, 'message': {'role': 'assistant', 'content': '{"action": "tool", "tool": "pdf_read", "args": {"pdf_path": "data/some_book.pdf"}}', 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 775, 'completion_tokens': 28, 'total_tokens': 803, 'cost': 0.00022175, 'is_byok': False, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0, 'video_tokens': 0}, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 0.00019375, 'upstream_inference_completions_cost': 2.8e-05}, 'completion_tokens_details': {'reasoning_tokens': 0, 'image_tokens': 0}}}
Executing Tool: pdf_read with Args {'pdf_path': 'data/some_book.pdf'}
Tool Result: ToolResult(artifact_path=None, result={'status': 'failed', 'content': 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

[Part 1/12] Ingested 26 chunks


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

[Part 2/12] Ingested 31 chunks


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

[Part 12/12] Ingested 1 chunks
Tool Result: ToolResult(artifact_path=None, result={'status': 'success', 'pdf_path': 'data/some_book.pdf', 'source': 'some_book'})
{'id': 'gen-1766824286-MvWUEUwyIvilm28UnOfG', 'provider': 'WandB', 'model': 'qwen/qwen3-235b-a22b-2507', 'object': 'chat.completion', 'created': 1766824286, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '{"action": "tool", "tool": "answer_question", "args": {"question": "Who tried to steal the stone in the book?"}}', 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 5676, 'completion_tokens': 32, 'total_tokens': 5708, 'cost': 0.0005708, 'is_byok': False, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0, 'video_tokens': 0}, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 0.0005676, 'upstream_inference_completions_cost': 3.2e-06}, 'completion_tokens_details': {'reaso

Batches: 100%|██████████| 1/1 [00:00<00:00, 15.04it/s]


{'id': 'gen-1766824288-thWgmVE4sgit1inLwn63', 'provider': 'WandB', 'model': 'qwen/qwen3-235b-a22b-2507', 'object': 'chat.completion', 'created': 1766824288, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': 'The person who tried to steal the Stone in the book was **Professor Quirrell**.\n\nThis is supported by the context on **Page 213**, where Dumbledore says:  \n> “Professor Quirrell did not manage to take it from you. I arrived in time to prevent that…”  \n\nThis clearly indicates that Quirrell attempted to steal the Stone.', 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 2808, 'completion_tokens': 76, 'total_tokens': 2884, 'cost': 0.0002884, 'is_byok': False, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0, 'video_tokens': 0}, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 0.0002808, 'upstream_inference_completions_cos

'Professor Quirrell tried to steal the stone.'

# 5. Longer Running coding Task

In [6]:
eval(
    "Load iris dataset from sklearn, Do a SVM and KNN comparison, visualize the confusion matrix and decision boundaries in PCA space, make sure all charts are labelled",
    12
)

Executing Tool: code_writer with Args {'task': 'Load the iris dataset from sklearn, perform SVM and KNN classification, and visualize the confusion matrix and decision boundaries in PCA space. Ensure all charts are labeled.', 'context': 'Import necessary libraries and load the iris dataset from sklearn. Perform SVM and KNN classification. Use PCA to reduce dimensions and visualize the confusion matrix and decision boundaries. Ensure all charts are labeled.'}
```python
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
import pandas as pd
import pickle
import os

# Load the iris dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target
target_names = iris.target_names

'The task is complete. The confusion matrices and decision boundaries in PCA space for both SVM and KNN models have been visualized and labeled.'